## Normalize DeepLabCut generated csv files

In [ ]:
!python ../Python_scripts/normalizeCSV.py

## Insert the normalized csv files into database

In [ ]:
import os
import psycopg2

# === CONFIG ===
normalized_dir = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered\NormalizedCSVs"
DB_NAME = "deeplabcut_db"
DB_USER = "postgres"
DB_PASSWORD = "1234"
DB_HOST = "localhost"
DB_PORT = "5432"

# === CONNECT TO DATABASE ===
conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
)
cursor = conn.cursor()

# === FETCH ALL COORD_PATH ENTRIES ===
cursor.execute("SELECT id, coord_path FROM dlc_files")
rows = cursor.fetchall()

# === PROCESS EACH ENTRY ===
updated = 0
for row in rows:
    id_, coord_path = row
    base_name = os.path.basename(coord_path)

    if "_normalized.csv" not in base_name:
        # Build expected normalized filename
        if base_name.endswith(".csv"):
            normalized_name = base_name.replace(".csv", "_normalized.csv")
            normalized_path = os.path.join(normalized_dir, normalized_name)

            if os.path.exists(normalized_path):
                cursor.execute(
                    "UPDATE dlc_files SET normalized_path = %s WHERE id = %s",
                    (normalized_path, id_)
                )
                updated += 1
            else:
                print(f"❗ Normalized file not found for: {base_name}")
        else:
            print(f"⚠️ Skipping non-CSV: {base_name}")

# === COMMIT AND CLOSE ===
conn.commit()
cursor.close()
conn.close()
print(f"✅ Done. Updated {updated} entries.")